In [1]:
import tensorflow as tf
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
# import data
drive.mount('/content/drive')
with open('/content/drive/My Drive/Colab Notebooks/projects/data.csv', 'r') as f: 
  data = np.genfromtxt(f, dtype='f4', delimiter=',')
samplenum = 5000
X = data[:samplenum,:-1]
y = data[:samplenum,-1]
print('# of samples = ', samplenum)
# partition into train, validate, test set at 60/20/20 division
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

Mounted at /content/drive
# of samples =  5000


All features

In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Normalization(),
    tf.keras.layers.Dense(10, input_dim=X.shape[1], activation = 'softplus'),
    tf.keras.layers.Dense(10, activation = 'softplus'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
net.compile(loss='MeanSquaredError', optimizer='adam', metrics=['MeanSquaredError'])
history = net.fit(X_train, y_train, epochs=50, verbose=0, batch_size=50, validation_data=(X_val,y_val))
score = net.evaluate(X_test, y_test,verbose=0)
print('Score = ', score[0])

Score =  0.001724247937090695


Subset selection: Forward stepwise selection 

In [ ]:
scores = np.ones((X.shape[1],X.shape[1]))*np.inf

for num in range(X.shape[1]):
  print('Now selecting the #',num,'significant feature.')
  net = tf.keras.models.Sequential([
      tf.keras.layers.Normalization(),
      tf.keras.layers.Dense(10, input_dim=num+1, activation = 'softplus'),
      tf.keras.layers.Dense(10, activation = 'softplus'),
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])
  net.compile(loss='MeanSquaredError', optimizer='adam', metrics=['MeanSquaredError'])
  for nextfeature in range(X.shape[1]):
    if nextfeature in np.argmin(scores[:num,:],axis=1):
      continue
    feature = np.append(np.argmin(scores[:num,:],axis=1),nextfeature)
    history = net.fit(X_train[:,feature], y_train, epochs=50, verbose=0, batch_size=50, validation_data=(X_val[:,feature],y_val))
    score = net.evaluate(X_test[:,feature], y_test,verbose=0)
    print('Feature', feature, '. Score = ', score[0])
    scores[num,nextfeature] = score[0]
  print('Feature', np.argmin(scores[:num+1,:],axis=1), 'has/have the most significant impact.')

Now selecting the # 0 significant feature.
Feature [0] . Score =  0.07098355144262314
Feature [1] . Score =  0.1153608039021492
Feature [2] . Score =  0.0448165237903595
Feature [3] . Score =  0.12435778230428696
Feature [4] . Score =  0.12851279973983765
Feature [5] . Score =  0.12543009221553802
Feature [6] . Score =  0.09412887692451477
Feature [7] . Score =  0.31687185168266296
Feature [8] . Score =  0.12299623340368271
Feature [9] . Score =  0.0655396580696106
Feature [10] . Score =  0.1306932121515274
Feature [2] has/have the most significant impact.
Now selecting the # 1 significant feature.
Feature [2 0] . Score =  0.002309653675183654
Feature [2 1] . Score =  0.037182752043008804
Feature [2 3] . Score =  0.044875048100948334
Feature [2 4] . Score =  0.04536878690123558
Feature [2 5] . Score =  0.044233329594135284
Feature [2 6] . Score =  0.03417905047535896
Feature [2 7] . Score =  0.037316806614398956
Feature [2 8] . Score =  0.04538632184267044
Feature [2 9] . Score =  0.01

Dongzheng Chen’s previous theoretical studies on this electronic problem indicate that Features [0,1,2] are the three major features.